# Contextual chatbot for financial services using Llama2 via SageMaker JumpStart and Amazon OpenSearch Serverless with Vector Engine

## Prerequisites

You will need these prerequisites in order to build the following context aware chatbot:
- An Amazon SageMaker Execution Role with IAM permission to access Amazon OpenSearch Serverless (aoss). The [in-line](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console) policy can be found [here](./IAM/sagemaker-execution-role-aoss-policy.yml). You can determine the correct SageMaker Role by running the `setup sagemaker session` cell below.
- An Amazon OpenSearch Serverless (aoss) Vector Engine. Setup instructions can be found [here](https://aws.amazon.com/blogs/big-data/introducing-the-vector-engine-for-amazon-opensearch-serverless-now-in-preview/).
- Add the SageMaker Execution Role to the Data Access Policy for Amazon OpenSearch Serverless (aoss). Instructions can be found [here](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-data-access.html#serverless-data-access-console).
- 2 x ml.g5.2xlarge instance types for SageMaker Endpoints. Instruction on increasing your service quota can be found [here](https://aws.amazon.com/getting-started/hands-on/request-service-quota-increase/).

## Install Required Python Libraries

    **IMPORTANT**
    1. Ensure you are running Pythin 3.10+
    1. Ensure you are using the Data Science 3.0 kernel

In [ ]:
#### Require python 3.10+
!python --version

Begin by installing the required python libraries.

In [ ]:
%pip install -U pip --quiet
%pip install --upgrade sagemaker --quiet 
%pip install langchain --quiet
%pip install opensearch-py --quiet
%pip install regex --quiet
%pip install tqdm --quiet
%pip install requests_aws4auth --quiet
%pip install pypdf

## Setup Environment

In [ ]:
# Setup SageMaker Session
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker import get_execution_role


sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
sm_execution_role = get_execution_role()

aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

print(sm_execution_role)

In [ ]:
# Import langchain 
from langchain.document_loaders import UnstructuredHTMLLoader,BSHTMLLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.vectorstores import OpenSearchVectorSearch
from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json


In [ ]:
llm_model = JumpStartModel(model_id = "meta-textgeneration-llama-2-7b-f")
llm_predictor = llm_model.deploy()
llm_endpoint_name=llm_predictor.endpoint_name
llm_endpoint_name

In [ ]:
embeddings_model = JumpStartModel(model_id = "huggingface-textembedding-gpt-j-6b-fp16")
embed_predictor = embeddings_model.deploy()
embeddings_endpoint_name = embed_predictor.endpoint_name
embeddings_endpoint_name

#### Ensure you update the variable below to reflect your enviroment 

In [ ]:
# Set variables for Amazon OpenSearch
_aoss_host="xxxx.us-east-1.aoss.amazonaws.com"
_aoss_index='your-index'

_aoss_host

## Chunk your Data and Load into Amazon OpenSearch

In this section we will chunk the data into smaller documents. Chunking is a technique for splitting large texts into smaller chunks. It is an important step as it optimizes the relevance of the search query for our RAG-model. Which in turn improves the quality of the chatbot. The chunk size is dependent on factors such as the document type and model used. We have selected a `chunk_size=1600` as this is the approximate size of a paragraph. As models improve, their context window size will increase which will allow for larger chunk sizes.

In [ ]:
loader = PyPDFLoader("./data/amazon-10k-2022.pdf")
data = loader.load()

In [ ]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

Next, we use LangChain `RecursiveCharacterTextSplitter` class to to chunk the document. 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1600, chunk_overlap=200)
docs = text_splitter.split_documents(data)

print (f'Now you have {len(docs)} documents')

In [ ]:
# Helper function to process document

import regex as re

def postproc(s):
    s = s.replace(u'\xa0', u' ') # no-break space 
    s = s.replace('\n', ' ') # new-line
    s = re.sub(r'\s+', ' ', s) # multiple spaces
    return s

In [ ]:
for doc in docs:
    doc.page_content = postproc(doc.page_content)

In the next step, we simply validate that document was chunked correctly by manually reviewing the first chunk.

In [ ]:
# Review the first document for correctness
docs[10]

In [ ]:
# Limit the number of total chunks to 1000
MAX_DOCS = 1000
if len(docs) > MAX_DOCS:
    docs = docs[:MAX_DOCS]

Next we extend the LangChain `SageMakerEndpointEmbeddings` Class to create a custom embeddings function that uses the `gpt-j-6b-fp16` SageMaker Endpoint you created earlier in this notebook.

In [ ]:
import time
import json
import logging
from typing import List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

logger = logging.getLogger(__name__)

# extend the SagemakerEndpointEmbeddings class from langchain to provide a custom embedding function
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(
        self, texts: List[str], chunk_size: int = 1
    ) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        st = time.time()
        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i:i + _chunk_size])
            results.extend(response)
        time_taken = time.time() - st
        logger.info(f"got results for {len(texts)} in {time_taken}s, length of embeddings list is {len(results)}")
        print(f"got results for {len(texts)} in {time_taken}s, length of embeddings list is {len(results)}")
        return results


# class for serializing/deserializing requests/responses to/from the embeddings model
class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:

        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode('utf-8') 

    def transform_output(self, output: bytes) -> str:

        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        if len(embeddings) == 1:
            return [embeddings[0]]
        return embeddings
    

def create_sagemaker_embeddings_from_js_model(embeddings_endpoint_name: str, aws_region: str) -> SagemakerEndpointEmbeddingsJumpStart:
    # all set to create the objects for the ContentHandler and 
    # SagemakerEndpointEmbeddingsJumpStart classes
    content_handler = ContentHandler()

    # note the name of the LLM Sagemaker endpoint, this is the model that we would
    # be using for generating the embeddings
    embeddings = SagemakerEndpointEmbeddingsJumpStart( 
        endpoint_name=embeddings_endpoint_name,
        region_name=aws_region, 
        content_handler=content_handler
    )
    return embeddings

We create the embeddings object and batch the creation of the document embeddings. These embeddinga are stored in Amazon OpenSearch using LangChain `OpenSearchVectorSearch`.


In [ ]:
embeddings = create_sagemaker_embeddings_from_js_model(embeddings_endpoint_name, aws_region)

In [ ]:
from tqdm import trange
from requests_aws4auth import AWS4Auth
from opensearchpy import RequestsHttpConnection

credentials = boto3.Session().get_credentials()

service="aoss"
region=aws_region

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

docsearch = OpenSearchVectorSearch.from_texts(
    texts = [d.page_content for d in docs],
    embedding=embeddings,
    opensearch_url=[{'host': _aoss_host, 'port': 443}],
    http_auth=awsauth,
    timeout = 300,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=_aoss_index
)

## Question Answering Over Documents 

So far, we have chunked a large document into smaller ones, created vector embedding and stored them in an Amazon OpenSearch Serverless with Vector engine. Now, we can answer questions regarding this document data.

Since we have created an index over the data, we can do a semantic search; this way only the most relevant documents required to answer the question are passed via the prompt to the Large Language Model (LLM). This allows you to save both time and money by not passing all the documents to the LLM.

The LLM used is `Llama2` via the SageMaker Endpoint created earlier.

We use LangChian **question_answering** `stuff` document chain type in this example. Further details on Document Chains can be found by visiting the [LangChain documentation, here](https://python.langchain.com/docs/modules/chains/document/)

In [ ]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

query = "what is the name of the Registrant’s principal executive officer??"
# query =  "Summarize the earnings report and also what year is the report for"
# query =  "what was North America's net sale ?"
# query= "summarize Risk factors section"

prompt_template = """Use the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": [
                [
                    {
                        "role": "system",
                        "content": prompt,
                    },
                    {"role": "user", "content": prompt},
                ],
            ],
            "parameters": {
                "max_new_tokens": 1000,
                "top_p": 0.9,
                "temperature": 0.6,
            },
        }
        input_str = json.dumps(
            payload,
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        content = response_json[0]["generation"]["content"]
        return content
    
content_handler = ContentHandler()

chain = load_qa_chain(
    llm=SagemakerEndpoint(
        endpoint_name=llm_endpoint_name,
        # credentials_profile_name="credentials-profile-name",
        region_name=aws_region,
        model_kwargs={"max_new_tokens": 300},
        endpoint_kwargs={"CustomAttributes": "accept_eula=true"},
        content_handler=content_handler,
    ),
    prompt=prompt,
)
sim_docs = docsearch.similarity_search(query, include_metadata=False)
# print(sim_docs)
chain({"input_documents": sim_docs, "question": query}, return_only_outputs=True)

## Clean Up

Delete the SageMaker Inference Endpoints that we created in this notebook to avoid incurring future costs. If you created an Amazon OpenSearch Serverless Collection for this example and no longer require it then delete it via the AWS Console.

In [ ]:
# Delete LLM
llm_predictor.delete_model()
llm_predictor.delete_predictor(delete_endpoint_config=True)

# Delete Embeddings Model
embed_predictor.delete_model()
embed_predictor.delete_predictor(delete_endpoint_config=True)

# Delete OpenSearch

## Conclusion

In this notebook, we used Retrieval Augmented Generation(RAG) as an optional approach to provide domain specific context to Large Language Models(LLM). We showed how to use SageMaker Jumpstart to easily build a RAG-based contextual chatbot for a financial services organization using Llama2 and Amazon OpenSearch Serverless as the Vector datastore. This method refines text generation using Llama2 by dynamically sourcing relevant context.